In [1]:
!pip install --upgrade pip
!pip install transformers
!pip install datasets>=1.18.3
!pip install librosa
!pip install jiwer
!pip install evaluate>=0.30



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main
!pip install huggingface_hub
!pip install -U datasets huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy

In [3]:
from scipy.io import wavfile
import scipy
from IPython.core.display import display
import os
import random
import IPython
import soundfile
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import json
import time
from huggingface_hub import notebook_login
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import load_dataset, DatasetDict, Audio, load_metric
from transformers import WhisperTokenizer, WhisperProcessor, WhisperFeatureExtractor
from jiwer import wer

from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

/tmp/ipykernel_34/4241458573.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [4]:
#utilities

notebook_login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_wav_duration(audio_array, sample_rate):
    length = len(audio_array) / sample_rate

    return int(length)

In [13]:
from datetime import datetime as dt
import time
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import soundfile
import librosa
import numpy as np


class WhisperModel:
  def __init__(self, check_point):
    ckpt = check_point # "openai/whisper-medium" #"openai/whisper-tiny" #"vasista22/whisper-hindi-medium"
    self.processor = WhisperProcessor.from_pretrained(ckpt)
    self.model = WhisperForConditionalGeneration.from_pretrained(ckpt,)
    self.model.generation_config.language = "en"
#     self.model.config.forced_decoder_ids = None
    self.model.to(device)
    self.initialize_supplements(check_point)

  def predict_with_whisper(self, audio_array, sample_rate):
    try:
      audio_array = librosa.resample(np.mean(audio_array, axis=1), orig_sr=sample_rate, target_sr=16000)
    except:
      audio_array = librosa.resample(audio_array, orig_sr=sample_rate, target_sr=16000)

    sample = {}
    sample["array"] = audio_array
    sample["sampling_rate"] = 16000
    input_features = self.processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").to(device).input_features
    # generate token ids

    predicted_ids = self.model.generate(input_features)

    transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)

    return transcription[0]

  def predict_single_example(self, path):
    signal, sr = soundfile.read(path, dtype="float32")
    return self.predict_with_whisper(signal, sr)

  def initialize_supplements(self, check_point):
    self.tokenizer = WhisperTokenizer.from_pretrained(check_point, language="en", task="transcribe")
    self.processor = WhisperProcessor.from_pretrained(check_point, language="en", task="transcribe")
    self.feature_extractor = WhisperFeatureExtractor.from_pretrained(check_point)

In [6]:
from transformers import AutoModelForCTC, Wav2Vec2Processor, Wav2Vec2ForCTC


class Wav2Vec2Model:
  def __init__(self, check_point):
    self.check_point = check_point
    self.model = Wav2Vec2ForCTC.from_pretrained(check_point)
    self.model.to(device)
    self.processor = Wav2Vec2Processor.from_pretrained(check_point)

  def predict_with_wav2vec2(self, audio_array, sample_rate):
    try:
      audio_array = librosa.resample(np.mean(audio_array, axis=1), orig_sr=sample_rate, target_sr=16000)
    except:
      audio_array = librosa.resample(audio_array, orig_sr=sample_rate, target_sr=16000)

    input_values = self.processor(audio_array, return_tensors="pt", padding="longest").input_values  # Batch size 1
    logits = self.model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = self.processor.batch_decode(predicted_ids)

    return transcription




In [7]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors'
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [8]:
import string
import re

class ASRDataset:
  def __init__(self, config):
    self.dataset_path, self.config = "yashtiwari/PaulMooney-Medical-ASR-Data", config
    self.train_data, self.validation_data, self.test_data = None, None, None
    self.data = DatasetDict()
    self.preprocessor = DataPreprocessing(self.config)
    self.data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=self.preprocessor.processor)

  def load_dataset(self):
    '''
    In original dataset test samples >>> train + valid
    '''
    self.data["train"] = load_dataset(self.dataset_path, split="test", use_auth_token=True)
    self.data["validation"] = load_dataset(self.dataset_path, split="validation", use_auth_token=True)
    self.data["test"] = load_dataset(self.dataset_path, split="train", use_auth_token=True)

    print(f"data is loaded successfully!! \n {self.data}")

  def preprocess_data(self):
    self.data = self.data.remove_columns(
    ["prompt", "speaker_id", "id"])
    self.data = self.data.cast_column("path", Audio(sampling_rate=16000))

    #action
    # self.data = self.data.map(self.preprocessor.remove_special_characters, num_proc=1)
    self.data = self.data.map(self.preprocessor.prepare_dataset, remove_columns = self.data.column_names["train"], num_proc=1)
    
    print(f"preprocessing completed !!!")
    print(f"dataset structure looks like: {self.data['train']}")

class DataPreprocessing:
  def __init__(self, config):
    self.config = config
    if config["name"] == "whisper":
      self.tokenizer = config["tokenizer"]
      self.processor = config["processor"]
      self.feature_extractor = config["feature_extractor"]
    else:
      self.processor = config["processor"]

  def prepare_dataset(self, batch):
#     if self.config["name"] == "whisper":
#       self.prepare_dataset_whipser(batch)
#     else:
#       self.prepare_dataset_wav2vec2(batch)
    # load and resample audio data from 48 to 16kHz
    audio = batch["path"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = self.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = self.tokenizer(batch["sentence"]).input_ids
    return batch

  def prepare_dataset_whipser(self, batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["path"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = self.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = self.tokenizer(batch["sentence"]).input_ids
    return batch

  def prepare_dataset_wav2vec2(self, batch):
    # load and resample audio data from 48 to 16kHz

    audio = batch["path"]


    # batched output is "un-batched" to ensure mapping is correct
    self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values
    batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values
    with self.processor.as_target_processor():
        batch["labels"] = self.processor(batch["sentence"]).input_ids

    return batch

  def remove_punctuation_and_lower(self, text):
    punctuation = re.sub(r"\'", r"", string.punctuation)
    # for i in range(len(texts)):
    return text.translate(str.maketrans("", "", punctuation)).upper()

  def remove_special_characters(self, batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

  def create_vocab_json(self):
    self.vocab_dict = {v: k for k, v in enumerate(list(self.vocab_set))}
    self.vocab_dict["|"] = self.vocab_dict.pop(" ")
    self.vocab_dict["[UNK]"] = len(self.vocab_dict)
    self.vocab_dict["[PAD]"] = len(self.vocab_dict)
    with open("./med_asr_vocab.json", "w") as f:
        json.dump(self.vocab_dict, f)



In [9]:

class Report:
  def __init__(self, whipser_config = None, wav2vec2_config = None, mode = "whisper"):

    #initialize models and necesssities
    self.dataset_path, self.mode = "yashtiwari/PaulMooney-Medical-ASR-Data", mode
    if mode == "whisper":
      self.config = whisper_config
      self.whisper_initialization(whipser_config)
    else:
      self.config = wav2vec2_config
      self.wav2vec2_initialization(wav2vec2_config)

    #initialize
    self.initialize_supplements()

  def whisper_initialization(self, whipser_config):
    self.whisper_config = whisper_config
    model_type, model_version = whipser_config["model_type"], whipser_config["model_version"]
    if model_type == "default":

      if model_version == "medium":
        check_point = "openai/whisper-medium"
      elif model_version == "tiny":
        check_point = "openai/whisper-tiny"
      elif model_version == "small":
        check_point = "openai/whisper-small"
      elif model_version == "large":
        check_point = "openai/whisper-large-v2"
      else:
        raise ValueError("Please input a valid model_version name from ['medium', 'tiny', 'large', 'small']")
    elif model_type == "fine-tuned":
      pass

    else:
        raise ValueError("Please input a valid model_type  from ['default', 'fine-tuned']")

    self.whisper_model = WhisperModel(check_point)
    self.whisper_config.update({"processor": self.whisper_model.processor, "feature_extractor": self.whisper_model.feature_extractor, "tokenizer": self.whisper_model.tokenizer})
    self.data, self.check_point = None, check_point


  def wav2vec2_initialization(self, wav2vec2_config):
    check_point = wav2vec2_config["check_point"]
    self.wav2vec2_model = Wav2Vec2Model(check_point)
    self.data, self.check_point = None, check_point

  def initialize_supplements(self):
    self.metric = load_metric("wer")

    #prepare dataset
    self.med_asr = ASRDataset(self.config)
    self.med_asr.load_dataset()
    self.med_asr.preprocess_data()

  def compute_metrics(self, pred):
      pred_ids = pred.predictions
      label_ids = pred.label_ids
      label_ids[label_ids == -100] = self.whisper_model.tokenizer.pad_token_id
      pred_str = self.whisper_model.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
      label_str = self.whisper_model.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
      wer = 100 * self.metric.compute(predictions=pred_str, references=label_str)

      return {"wer": wer}

  def whisper_predictions(self, batch):
    generated_tokens = (
        self.whisper_model.model.generate(
            input_features=batch["input_features"].to(device),
            decoder_input_ids=batch["labels"][:, :4].to(device),
            max_new_tokens=255,
        )
        .cpu()
        .numpy()
    )
    labels = batch["labels"].cpu().numpy()
    labels = np.where(labels != -100, labels, self.whisper_model.tokenizer.pad_token_id)
    decoded_preds = self.whisper_model.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    decoded_labels = self.whisper_model.tokenizer.batch_decode(labels, skip_special_tokens=True)

    return decoded_preds, decoded_labels

  def process_data_whisper(self, split):
    eval_dataloader = DataLoader(self.med_asr.data[split], batch_size=8, collate_fn=self.med_asr.data_collator)

    self.whisper_model.model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.cuda.amp.autocast():
            with torch.no_grad():
                generated_tokens = (
                    self.whisper_model.model.generate(
                        input_features=batch["input_features"].to(device),
                        decoder_input_ids=batch["labels"][:, :4].to(device),
                        max_new_tokens=255,
                    )
                    .cpu()
                    .numpy()
                )
                labels = batch["labels"].cpu().numpy()
                labels = np.where(labels != -100, labels, self.whisper_model.tokenizer.pad_token_id)
                decoded_preds = self.whisper_model.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                decoded_labels = self.whisper_model.tokenizer.batch_decode(labels, skip_special_tokens=True)
                self.metric.add_batch(
                    predictions=decoded_preds,
                    references=decoded_labels,
                )
        del generated_tokens, labels, batch
        gc.collect()
    wer = 100 * self.metric.compute()
    return wer

  def process_data_wav2vec2(self, split):
    eval_dataloader = DataLoader(self.med_asr.data[split], batch_size=8, collate_fn=self.med_asr.data_collator)
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.cuda.amp.autocast():
          with torch.no_grad():
            input_values = torch.tensor(batch["input_values"], device=device).unsqueeze(0)
            logits = model(input_values).logits

            pred_ids = torch.argmax(logits, dim=-1)
            decoded_preds = self.model.processor.batch_decode(pred_ids)[0]
            decoded_labels = self.model.processor.decode(batch["labels"], group_tokens=False)
            self.metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
        del logits, pred_ids, batch
        gc.collect()
    wer = 100 * self.metric.compute()
    return wer

  def process_data(self, split = "test"):
    if self.mode == "whisper":
      wer = self.process_data_whisper(split)
    else:
      wer = self.process_data_wav2vec2(split)

    print(f"Data processed compeletely!!!")
    return wer 
    

  def generate_report(self):

    validation_wer = self.process_data("validation")
    test_wer = self.process_data("test")

    print("="*30)
    print(f"dataset path: {self.dataset_path}")
    print(f"-"*30)
    print(f"model check point: {self.check_point}")
    print(f"-"*30)
    print(f"validation WER: {validation_wer} ")
    print(f"-"*30)
    print(f"test WER: {test_wer} ")
    print("="*30)


In [14]:
model = WhisperModel("openai/whisper-small")
path = "/kaggle/input/random/voice.wav"
model.predict_single_example(path)

' Jack Pepsi CD'

In [10]:
if __name__ == "__main__":
  whisper_config = {"name": "whisper", "model_type": "default", "model_version": "small"}
  report = Report(whisper_config)
  report.generate_report()

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

preprocessing completed !!!
dataset structure looks like: Dataset({
    features: ['input_features', 'labels'],
    num_rows: 5895
})


  0%|          | 0/49 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:1286: FutureWarning: You have provided `decoder_input_ids` which will overwrite the `init_tokens` [50258, 50259, 50359, 50363]. This might lead to unexpected behavior. Passing `decoder_input_ids` is deprecated and will be removed in v4.39. Consider passing `prompt_ids` instead.
  warnings.warn(
100%|██████████| 49/49 [02:17<00:00,  2.80s/it]


Data processed compeletely!!!


100%|██████████| 48/48 [02:12<00:00,  2.76s/it]

Data processed compeletely!!!
dataset path: yashtiwari/PaulMooney-Medical-ASR-Data
------------------------------
model check point: openai/whisper-small
------------------------------
validation WER: 28.64476386036961 
------------------------------
test WER: 17.30629978276611 
